In [2]:
from datetime import datetime
import threading
import time
import re


In [3]:
# Here we are initailizing the variables to store the reminders
reminders = []

# Dictionary to store the converstation context for interactive reminder settting
conversation_context = {}

def normalize_time(user_input):
    """
    Converts unconventional time formats into a standard 12-hour HH:MM AM/PM format.
    Example Inputs:
    - '914pm' -> '09:14 PM'
    - '915pm' -> '09:15 PM'
    - '9pm' -> '09:00 PM'
    """
    match = re.match(r'(\d{1,4})(AM|PM|am|pm)?', user_input, re.IGNORECASE)
    if not match:
        return None
    
    raw_time = match.group(1)
    period = match.group(2).upper() if match.group(2) else "PM"  # Default to PM if not specified

    if len(raw_time) == 1 or len(raw_time) == 2:  # '9' or '12'
        hour = int(raw_time)
        minute = 0
    elif len(raw_time) == 3:  # '914' -> 9:14
        hour = int(raw_time[0])
        minute = int(raw_time[1:])
    elif len(raw_time) == 4:  # '0915' -> 9:15
        hour = int(raw_time[:2])
        minute = int(raw_time[2:])
    else:
        return None  

    # Adjust minutes
    while minute >= 60:
        hour += 1
        minute -= 60

    if hour > 12:
        hour %= 12
    if hour == 0:
        hour = 12

    try:
        normalized_time = datetime.strptime(f"{hour}:{minute:02d} {period}", "%I:%M %p")
        return normalized_time.strftime("%I:%M %p")
    except ValueError:
        return None



In [ ]:
from datetime import datetime
import threading
import time
import re

# Global dictionary to store active reminders
reminders = []
conversation_context = {}

def normalize_time(user_input):
    """
    Normalize unconventional time inputs into a valid 12-hour HH:MM AM/PM format.
    For example:
    - '914pm' -> '09:14 PM'
    - '915pm' -> '09:15 PM'
    - '9pm' -> '09:00 PM'
    """
    # Extract numbers and AM/PM
    match = re.match(r'(\d{1,4})(AM|PM|am|pm)?', user_input, re.IGNORECASE)
    if not match:
        return None  # Invalid input
    
    raw_time = match.group(1)  # Extract the numeric part
    period = match.group(2).upper() if match.group(2) else "PM"  # Default to PM if not provided

    # Handle different lengths of the raw_time
    if len(raw_time) == 1 or len(raw_time) == 2:  # e.g., '9' or '12'
        hour = int(raw_time)
        minute = 0
    elif len(raw_time) == 3:  # e.g., '914' -> hour: 9, minute: 14
        hour = int(raw_time[0])
        minute = int(raw_time[1:])
    elif len(raw_time) == 4:  # e.g., '0915' or '915' -> hour: 9, minute: 15
        hour = int(raw_time[:2])
        minute = int(raw_time[2:])
    else:
        return None  # Invalid input length

    # Adjust hour and minute for valid time ranges
    while minute >= 60:
        hour += 1
        minute -= 60

    # Convert to 12-hour format
    if hour > 12:
        hour %= 12
    if hour == 0:
        hour = 12

    # Format into HH:MM AM/PM
    try:
        normalized_time = datetime.strptime(f"{hour}:{minute:02d} {period}", "%I:%M %p")
        return normalized_time.strftime("%I:%M %p")
    except ValueError:
        return None  # Invalid input after adjustments

# Reminder Handler
def handle_reminder(user_input):
    """
    Handles setting reminders interactively with the user.
    """
    global conversation_context

    # If there's a pending_action in conversation_context, continue that flow
    if "pending_action" in conversation_context:
        action = conversation_context.pop("pending_action")
        if action == "set_reminder_time":
            # Normalize time input
            normalized_time = normalize_time(user_input)
            if not normalized_time:
                return "Invalid time format. Please provide the time in HH:MM AM/PM format (e.g., 02:30 PM)."
            conversation_context["reminder_time"] = normalized_time
            conversation_context["pending_action"] = "set_reminder_message"
            return "What should I remind you about?"
        elif action == "set_reminder_message":
            # Now we have time and message, we can set the reminder
            reminder_time = conversation_context.pop("reminder_time")
            reminder_message = user_input
            set_reminder(reminder_time, reminder_message)
            return f"Reminder set for {reminder_time} with message: '{reminder_message}'"
    else:
        # Start the reminder setting process
        conversation_context["pending_action"] = "set_reminder_time"
        return "What time should I set the reminder for? (e.g., 02:30 PM)"

def set_reminder(reminder_time, message):
    """
    Create a background thread that triggers at the exact reminder_time (12-hour format).
    """
    def reminder_thread():
        while True:
            current_time = datetime.now().strftime("%I:%M %p")  # 12-hour format
            if current_time == reminder_time:
                print(f"\nReminder: {message}")
                break
            time.sleep(1)

    reminders.append({"time": reminder_time, "message": message})
    threading.Thread(target=reminder_thread, daemon=True).start()

# Test Reminder Functionality
if __name__ == "__main__":
    conversation_context = {}  # Initialize global conversation context
    print("Welcome to the Reminder Module!")
    print("You can interactively set reminders.")

    while True:
        user_input = input("You: ").strip()
        if user_input.lower() in ["exit", "quit"]:
            print("Goodbye!")
            break
        response = handle_reminder(user_input)
        print(f"Assistant: {response}")

Welcome to the Reminder Module!
You can interactively set reminders.
Assistant: What time should I set the reminder for? (e.g., 02:30 PM)
Assistant: Invalid time format. Please provide the time in HH:MM AM/PM format (e.g., 02:30 PM).
Assistant: What time should I set the reminder for? (e.g., 02:30 PM)
Assistant: Invalid time format. Please provide the time in HH:MM AM/PM format (e.g., 02:30 PM).
Assistant: What time should I set the reminder for? (e.g., 02:30 PM)
Assistant: What should I remind you about?
Assistant: Reminder set for 02:00 PM with message: 'Buy apples for DevAnand'
Assistant: What time should I set the reminder for? (e.g., 02:30 PM)
Assistant: Invalid time format. Please provide the time in HH:MM AM/PM format (e.g., 02:30 PM).
Assistant: What time should I set the reminder for? (e.g., 02:30 PM)
Assistant: Invalid time format. Please provide the time in HH:MM AM/PM format (e.g., 02:30 PM).
Assistant: What time should I set the reminder for? (e.g., 02:30 PM)
Assistant: In